In [ ]:
pip install gradio opencv-python numpy


In [ ]:
import gradio as gr
import cv2
import numpy as np

# ======================= Filters =======================

def is_grayscale(img):
    return len(img.shape) == 2 or (len(img.shape) == 3 and img.shape[2] == 1 or np.array_equal(img[..., 0], img[..., 1]) and np.array_equal(img[..., 1], img[..., 2]))

def to_gray(img):
    if is_grayscale(img):
        return img if len(img.shape) == 2 else cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def add_noise(img):
    return np.clip(img + np.random.normal(0, 20, img.shape), 0, 255).astype(np.uint8)

def salt_pepper_noise(img):
    out = np.copy(img)
    amount = 0.02
    num_salt = np.ceil(amount * img.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in img.shape[:2]]
    out[coords[0], coords[1]] = 255

    num_pepper = np.ceil(amount * img.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in img.shape[:2]]
    out[coords[0], coords[1]] = 0
    return out

def remove_noise(img): return cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
def mean_filter(img): return cv2.blur(img, (5, 5))
def median_filter(img): return cv2.medianBlur(img, 5)
def gaussian_filter(img): return cv2.GaussianBlur(img, (5, 5), 0)
def erosion(img): return cv2.erode(img, np.ones((5,5),np.uint8), iterations=1)
def dilation(img): return cv2.dilate(img, np.ones((5,5),np.uint8), iterations=1)
def opening(img): return cv2.morphologyEx(img, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
def closing(img): return cv2.morphologyEx(img, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))

def boundary_extraction(img):
    gray = to_gray(img)
    erosion_img = cv2.erode(gray, np.ones((3,3), np.uint8), iterations=1)
    boundary = cv2.subtract(gray, erosion_img)
    return cv2.cvtColor(boundary, cv2.COLOR_GRAY2BGR)

def region_filling(img):
    gray = to_gray(img)
    _, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    im_floodfill = th.copy()
    h, w = th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0,0), 255)
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    filled = th | im_floodfill_inv
    return cv2.cvtColor(filled, cv2.COLOR_GRAY2BGR)

def global_threshold(img):
    gray = to_gray(img)
    _, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    return cv2.cvtColor(th, cv2.COLOR_GRAY2BGR)

def adaptive_threshold(img):
    gray = to_gray(img)
    th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)
    return cv2.cvtColor(th, cv2.COLOR_GRAY2BGR)

def otsu_threshold(img):
    gray = to_gray(img)
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return cv2.cvtColor(th, cv2.COLOR_GRAY2BGR)

def hough_transform(img):
    gray = to_gray(img)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=50, maxLineGap=10)
    result = img.copy()
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line[0]
            cv2.line(result, (x1,y1), (x2,y2), (0,255,0), 2)
    return result

def watershed_segment(img):
    gray = to_gray(img)
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2,5)
    _, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers+1
    markers[unknown==255] = 0
    markers = cv2.watershed(img, markers)
    img[markers == -1] = [255,0,0]
    return img

def edge_detection(img):
    gray = to_gray(img)
    edges = cv2.Canny(gray, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

# ========== Filters Map ==========

filters_map = {
    "Add Noise": add_noise,
    "Salt & Pepper Noise": salt_pepper_noise,
    "Remove Noise": remove_noise,
    "Mean Filter": mean_filter,
    "Median Filter": median_filter,
    "Gaussian Filter": gaussian_filter,
    "Erosion": erosion,
    "Dilation": dilation,
    "Opening": opening,
    "Closing": closing,
    "Boundary Extraction": boundary_extraction,
    "Region Filling": region_filling,
    "Global Threshold": global_threshold,
    "Adaptive Threshold": adaptive_threshold,
    "Otsu Threshold": otsu_threshold,
    "Hough Transform": hough_transform,
    "Watershed": watershed_segment,
    "Edges": edge_detection
}

# ========== Apply Selected Filters ==========

def apply_selected_filters(img, filters):
    if img is None or not filters:
        return img
    output = img.copy()
    for f in filters:
        func = filters_map.get(f)
        if func:
            output = func(output)
    return output

# ========== Save Image ==========

def save_image(img):
    path = "filtered_image.png"
    cv2.imwrite(path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    return path

# ========== Gradio Interface ==========

with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue")) as demo:
    gr.Markdown("## 🎨 Interactive Image Filter Application")

    selected_filters = gr.State([])
    button_states = gr.State({})

    with gr.Row():
        input_img = gr.Image(type="numpy", label="Original Image")
        output_img = gr.Image(type="numpy", label="Filtered Image")

    gr.Markdown("### 🧰 Choose Filters")
    filter_buttons = {}
    with gr.Row():
        for name in filters_map:
            btn = gr.Button(value=name, elem_id=f"btn_{name}")
            filter_buttons[name] = btn

    selected_text = gr.Textbox(label="Selected Filters", interactive=False)

    with gr.Row():
        apply_btn = gr.Button("✅ Apply All Filters")
        reset_btn = gr.Button("🔁 Reset All")
        clear_btn = gr.Button("🧹 Clear Image & Filters")

    download_file = gr.File(label="📥 Download Image")

    def toggle_filter(name, current_filters, btn_states):
        new_filters = current_filters.copy()
        new_states = btn_states.copy()
        if name in new_filters:
            new_filters.remove(name)
            new_states[name] = "secondary"
        else:
            new_filters.append(name)
            new_states[name] = "primary"
        return new_filters, ", ".join(new_filters), new_states

    for fname, btn in filter_buttons.items():
        btn.click(fn=toggle_filter,
                  inputs=[gr.Text(fname, visible=False), selected_filters, button_states],
                  outputs=[selected_filters, selected_text, button_states])

    def update_button_colors(btn_states):
        updates = {}
        for name, variant in btn_states.items():
            updates[filter_buttons[name]] = gr.update(variant=variant)
        return updates

    button_states.change(fn=update_button_colors, inputs=button_states,
                         outputs=[*filter_buttons.values()])

    apply_btn.click(fn=apply_selected_filters, inputs=[input_img, selected_filters], outputs=output_img)
    apply_btn.click(fn=save_image, inputs=output_img, outputs=download_file)

    def reset_all(img):
        return img, [], "", {name: "secondary" for name in filters_map}

    reset_btn.click(fn=reset_all, inputs=input_img,
                    outputs=[output_img, selected_filters, selected_text, button_states])

    def clear_all():
        return None, None, [], "", {name: "secondary" for name in filters_map}

    clear_btn.click(fn=clear_all,
                    outputs=[input_img, output_img, selected_filters, selected_text, button_states])


demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://70acdba5caa7251d3d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# import gradio as gr
# import cv2
# import numpy as np

# # ======================= Filters =======================
# def add_noise(img):
#     return np.clip(img + np.random.normal(0, 20, img.shape), 0, 255).astype(np.uint8)

# def salt_pepper_noise(img):
#     out = np.copy(img)
#     amount = 0.02
#     num_salt = np.ceil(amount * img.size * 0.5)
#     coords = [np.random.randint(0, i - 1, int(num_salt)) for i in img.shape[:2]]
#     out[coords[0], coords[1]] = 255

#     num_pepper = np.ceil(amount * img.size * 0.5)
#     coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in img.shape[:2]]
#     out[coords[0], coords[1]] = 0
#     return out

# def remove_noise(img): return cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
# def mean_filter(img): return cv2.blur(img, (5, 5))
# def median_filter(img): return cv2.medianBlur(img, 5)
# def gaussian_filter(img): return cv2.GaussianBlur(img, (5, 5), 0)
# def erosion(img): return cv2.erode(img, np.ones((5,5),np.uint8), iterations=1)
# def dilation(img): return cv2.dilate(img, np.ones((5,5),np.uint8), iterations=1)
# def opening(img): return cv2.morphologyEx(img, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
# def closing(img): return cv2.morphologyEx(img, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))

# def boundary_extraction(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     erosion_img = cv2.erode(gray, np.ones((3,3), np.uint8), iterations=1)
#     boundary = cv2.subtract(gray, erosion_img)
#     return cv2.cvtColor(boundary, cv2.COLOR_GRAY2BGR)

# def region_filling(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
#     im_floodfill = th.copy()
#     h, w = th.shape[:2]
#     mask = np.zeros((h+2, w+2), np.uint8)
#     cv2.floodFill(im_floodfill, mask, (0,0), 255)
#     im_floodfill_inv = cv2.bitwise_not(im_floodfill)
#     filled = th | im_floodfill_inv
#     return cv2.cvtColor(filled, cv2.COLOR_GRAY2BGR)

# def global_threshold(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
#     return cv2.cvtColor(th, cv2.COLOR_GRAY2BGR)

# def adaptive_threshold(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                cv2.THRESH_BINARY, 11, 2)
#     return cv2.cvtColor(th, cv2.COLOR_GRAY2BGR)

# def otsu_threshold(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     return cv2.cvtColor(th, cv2.COLOR_GRAY2BGR)

# def hough_transform(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     edges = cv2.Canny(gray, 50, 150)
#     lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=50, maxLineGap=10)
#     result = img.copy()
#     if lines is not None:
#         for line in lines:
#             x1,y1,x2,y2 = line[0]
#             cv2.line(result, (x1,y1), (x2,y2), (0,255,0), 2)
#     return result

# def watershed_segment(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
#     kernel = np.ones((3,3),np.uint8)
#     opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=2)
#     sure_bg = cv2.dilate(opening, kernel, iterations=3)
#     dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2,5)
#     _, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
#     sure_fg = np.uint8(sure_fg)
#     unknown = cv2.subtract(sure_bg,sure_fg)
#     _, markers = cv2.connectedComponents(sure_fg)
#     markers = markers+1
#     markers[unknown==255] = 0
#     markers = cv2.watershed(img, markers)
#     img[markers == -1] = [255,0,0]
#     return img

# # ========== Filters Map ==========
# filters_map = {
#     "Add Noise": add_noise,
#     "Salt & Pepper Noise": salt_pepper_noise,
#     "Remove Noise": remove_noise,
#     "Mean Filter": mean_filter,
#     "Median Filter": median_filter,
#     "Gaussian Filter": gaussian_filter,
#     "Erosion": erosion,
#     "Dilation": dilation,
#     "Opening": opening,
#     "Closing": closing,
#     "Boundary Extraction": boundary_extraction,
#     "Region Filling": region_filling,
#     "Global Threshold": global_threshold,
#     "Adaptive Threshold": adaptive_threshold,
#     "Otsu Threshold": otsu_threshold,
#     "Hough Transform": hough_transform,
#     "Watershed": watershed_segment
# }

# # ========== Apply Selected Filters ==========
# def apply_selected_filters(img, filters):
#     if img is None or not filters:
#         return img
#     output = img.copy()
#     for f in filters:
#         func = filters_map.get(f)
#         if func:
#             output = func(output)
#     return output

# # ========== Save Image ==========
# def save_image(img):
#     path = "filtered_image.png"
#     cv2.imwrite(path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
#     return path

# # ========== Gradio Interface ==========
# with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue")) as demo:
#     gr.Markdown("## 🎨 Interactive Image Filter Application")

#     selected_filters = gr.State([])
#     button_states = gr.State({})

#     with gr.Row():
#         input_img = gr.Image(type="numpy", label="Original Image")
#         output_img = gr.Image(type="numpy", label="Filtered Image")

#     gr.Markdown("### 🧰 Choose Filters")
#     filter_buttons = {}
#     with gr.Row():
#         for name in filters_map:
#             btn = gr.Button(value=name, elem_id=f"btn_{name}")
#             filter_buttons[name] = btn

#     selected_text = gr.Textbox(label="Selected Filters", interactive=False)

#     with gr.Row():
#         apply_btn = gr.Button("✅ Apply All Filters")
#         reset_btn = gr.Button("🔁 Reset All")
#         clear_btn = gr.Button("🧹 Clear Image & Filters")

#     download_file = gr.File(label="📥 Download Image")

#     # ---------- Toggle Button ----------
#     def toggle_filter(name, current_filters, btn_states):
#         new_filters = current_filters.copy()
#         new_states = btn_states.copy()
#         if name in new_filters:
#             new_filters.remove(name)
#             new_states[name] = "secondary"
#         else:
#             new_filters.append(name)
#             new_states[name] = "primary"
#         return new_filters, ", ".join(new_filters), new_states

#     for fname, btn in filter_buttons.items():
#         btn.click(fn=toggle_filter,
#                   inputs=[gr.Text(fname, visible=False), selected_filters, button_states],
#                   outputs=[selected_filters, selected_text, button_states])

#     def update_button_colors(btn_states):
#         updates = {}
#         for name, variant in btn_states.items():
#             updates[filter_buttons[name]] = gr.update(variant=variant)
#         return updates

#     button_states.change(fn=update_button_colors, inputs=button_states,
#                          outputs=[*filter_buttons.values()])

#     apply_btn.click(fn=apply_selected_filters, inputs=[input_img, selected_filters], outputs=output_img)
#     apply_btn.click(fn=save_image, inputs=output_img, outputs=download_file)

#     def reset_all(img):
#         return img, [], "", {name: "secondary" for name in filters_map}

#     reset_btn.click(fn=reset_all, inputs=input_img,
#                     outputs=[output_img, selected_filters, selected_text, button_states])

#     def clear_all():
#         return None, None, [], "", {name: "secondary" for name in filters_map}

#     clear_btn.click(fn=clear_all,
#                     outputs=[input_img, output_img, selected_filters, selected_text, button_states])

# demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9fa05fb08b3520192c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
